In [ ]:
pip install pyspark

In [1]:
import configparser
from datetime import datetime
import os
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import datetime
import pandas
import pyspark.sql.functions as F

In [2]:
spark = SparkSession \
        .builder \
        .appName("Data lake project") \
        .getOrCreate()

In [3]:
# get filepath to song data file
filepath = "data/song-data/song_data"

In [4]:
# get all files matching extension from directory
song_data = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*.json'))
    for f in files :
        song_data.append(os.path.abspath(f))

In [5]:
# read song data file
dfsong = spark.read.json(song_data)


In [6]:
dfsong.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [7]:
print(dfsong.count())

74


In [8]:
dfsong.show(10)

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARDR4AC1187FB371A1|           null|                    |            null|Montserrat Caball...|511.16363|        1|SOBAYLL12A8C138AF9|Sono andati? Fing...|   0|
|AREBBGV1187FB523D2|           null|         Houston, TX|            null|Mike Jones (Featu...|173.66159|        1|SOOLYAZ12A6701F4A6|Laws Patrolling (...|   0|
|ARMAC4T1187FB3FA4C|       40.82624|   Morris Plains, NJ|       -74.47995|The Dillinger Esc...|207.77751|        1|SOBBUGU12A8C13E95D|Setting Fire to S...|2004|
|ARPBNLO1187FB3D52F|       40.7145

In [9]:
filepath2 = "./data/log-data"

In [10]:
# get all files matching extension from directory
log_data = []
for root, dirs, files in os.walk(filepath2):
    files = glob.glob(os.path.join(root,'*.json'))
    for f in files :
        log_data.append(os.path.abspath(f))

In [11]:


# read log data file
dflog = spark.read.json(log_data)


In [12]:
print(dflog.count())

8056


In [13]:
dflog.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [14]:
dflog.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


In [15]:
dflog.describe("artist").show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|              6820|
|   mean|             266.5|
| stddev|109.00229355385143|
|    min|               !!!|
|    max|   ÃÂtienne Daho|
+-------+------------------+



In [16]:
dflog.select("page").dropDuplicates().sort("page").show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [17]:
dflog.show()

+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|      auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|            Harmonia| Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|         The Prodigy| Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Su

In [18]:
dflog = dflog.withColumn('start_time',( (dflog.ts.cast('float')/1000).cast("timestamp")) )
dflog = dflog.select(
                    F.col("start_time").alias("start_time"),
                    F.hour("start_time").alias('hour'),
                    F.dayofmonth("start_time").alias('day'),
                    F.weekofyear("start_time").alias('week'),
                    F.month("start_time").alias('month'), 
                    F.year("start_time").alias('year'), 
                    F.date_format(F.col("start_time"), "E").alias("weekday")
                )

In [19]:
dflog.show(2)

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:29:...|   0| 15|  46|   11|2018|    Thu|
|2018-11-15 00:40:...|   0| 15|  46|   11|2018|    Thu|
+--------------------+----+---+----+-----+----+-------+
only showing top 2 rows



In [20]:
# extract columns to create time table
time_table = dflog.select(dflog['start_time'], dflog['hour'], dflog['day'], dflog['week'], dflog['month'],dflog['year'],\
                              dflog['weekday']).dropDuplicates()

In [21]:
time_table.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: string (nullable = true)



In [22]:
time_table.show(5)

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 18:00:...|  18| 15|  46|   11|2018|    Thu|
|2018-11-15 20:04:...|  20| 15|  46|   11|2018|    Thu|
|2018-11-21 04:36:...|   4| 21|  47|   11|2018|    Wed|
|2018-11-21 05:11:...|   5| 21|  47|   11|2018|    Wed|
|2018-11-21 08:36:...|   8| 21|  47|   11|2018|    Wed|
+--------------------+----+---+----+-----+----+-------+
only showing top 5 rows



In [ ]:
dfsongplay = dflog.join(dfsong, dflog.artist == dfsong.artist_name, "inner").\
select(dflog['start_time'], dflog['userId'], dflog['level'], dfsong['song_id'], dfsong['artist_id'], dflog['sessionId'],dflog['location'],dflog['userAgent'],
      ).dropDuplicates()

In [ ]:
dfuser = dflog.join(dfsong,dflog.artist == dfsong.artist_name, "inner").\
select(F.to_timestamp(dflog['ts'] / 1000).alias('start_time'), dflog['userId'], dflog['level'], dfsong['song_id'], dfsong['artist_id'], dflog['sessionId'],dflog['location'],dflog['userAgent'],
      ).dropDuplicates()

In [ ]:
dfuser.limit(5).toPandas()    

In [ ]:
dfsong.createOrReplaceTempView("staging_songs")
dflog.createOrReplaceTempView("staging_logs")

In [ ]:
songplay = spark.sql("""SELECT  DISTINCT timestamp, start_time, 
        user_id, 
        level,
        song_id, 
        artist_id, 
        session_id, 
        location, 
        user_agent

FROM staging_logs
JOIN staging_songs ON (staging_logs.artist = staging_songs.artist_name )
AND staging_logs.page  =  'NextSong'""") \
  .show(truncate=False)

In [ ]:
songplay = spark.sql("""SELECT  DISTINCT timestamp 'epoch' + ts/1000 * interval '1 second' AS start_time, 
        user_id, 
        level,
        song_id, 
        artist_id, 
        session_id, 
        location, 
        user_agent

FROM staging_logs
JOIN staging_songs ON (staging_logs.artist = staging_songs.artist_name )
AND staging_logs.page  =  'NextSong'""") \
  .show(truncate=False)

In [ ]:
dfsongplay.toPandas()